In [6]:
# required packages
#!pip install pandas
# hoi comment
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import clear_output
import tensorflow as tf
import cv2 as cv
from deepface import DeepFace

datapath = 'data/real_and_fake_face/'
print(datapath)

Directory  /home/cas /.deepface created
Directory  /home/cas /.deepface/weights created
data/real_and_fake_face/


In [6]:
#read the file names
# and group them by feature into a dataframe
f_temp = []
for path, subdirs, files in os.walk(datapath):
    for name in files:
        full_path = os.path.join(path, name)
        if name.startswith('real'):
            f_temp.append([full_path, 'real', 'real', 0, 0, 0, 0])
        elif name.startswith('easy') or name.startswith('mid') or name.startswith('hard'):
            split_name = name.split('_')
            label = 'fake'
            category = split_name[0]
            annotation = split_name[2]
            annotation = annotation.split('.')[0]
            if len(annotation) == 4:
                f_temp.append([full_path, label, category, annotation[0], annotation[1], annotation[2], annotation[3]])                
            else:
                print(f"ERROR image {name}")
data = pd.DataFrame(f_temp, columns=['filename', 'label','category','left_eye','right_eye','nose','mouth'])
print("amount of images: ", data.shape[0])
print(f"real images: {data[data['label']=='real'].shape[0]}")
print(f"fake images: {data[data['label']=='fake'].shape[0]}")
print(f"easy images: {data[data['category']=='easy'].shape[0]}")
print(f"mid images: {data[data['category']=='mid'].shape[0]}")
print(f"hard images: {data[data['category']=='hard'].shape[0]}")




amount of images:  2041
real images: 1081
fake images: 960
easy images: 240
mid images: 480
hard images: 240


In [9]:
#verify the deepface model
verification = DeepFace.verify("data/real_and_fake_face/training_real/real_00001.jpg", "data/real_and_fake_face/training_real/real_00002.jpg")
print(verification)

{'verified': True, 'distance': 0.21908850922038914, 'threshold': 0.4, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 146, 'y': 140, 'w': 428, 'h': 428}, 'img2': {'x': 80, 'y': 140, 'w': 436, 'h': 436}}, 'time': 0.71}


In [18]:
#find a face in a dataset
image_real = cv.imread("data/real_and_fake_face/training_real/real_00002.jpg")
db_path = "data/real_and_fake_face/training_fake/"
enforce_detection = False
x = DeepFace.find(img_path = image_real, db_path = db_path, enforce_detection = enforce_detection)
print(x)

Finding representations: 100%|██████████| 960/960 [04:39<00:00,  3.43it/s]


Representations stored in data/real_and_fake_face/training_fake//representations_vgg_face.pkl file.Please delete this file when you add new identities in your database.
find function lasts  281.11802649497986  seconds


In [20]:
#analyze face for features
features = ['age','gender','emotion','race']
x = DeepFace.analyze("data/real_and_fake_face/training_real/real_00002.jpg", actions = features)
print(x)

Action: race: 100%|██████████| 4/4 [00:00<00:00,  6.43it/s]   

[{'age': 33, 'region': {'x': 80, 'y': 140, 'w': 436, 'h': 436}, 'gender': {'Woman': 96.71955704689026, 'Man': 3.2804474234580994}, 'dominant_gender': 'Woman', 'emotion': {'angry': 0.006737637810951434, 'disgust': 1.4480664428770946e-14, 'fear': 99.99027252313226, 'happy': 0.0005937521311131792, 'sad': 0.0018653215697746156, 'surprise': 7.018928458833224e-05, 'neutral': 0.00045881828542566807}, 'dominant_emotion': 'fear', 'race': {'asian': 24.265503970920566, 'indian': 6.425423154184442, 'black': 2.348557020895836, 'white': 34.15506074964276, 'middle eastern': 11.184428044029707, 'latino hispanic': 21.62102054106944}, 'dominant_race': 'white'}]
